In [ ]:
##-----------------------------------------------------------------------------------------------##
##
## ESTRATÉGIAS DE INVESTIMENTO NO MERCADO DE AÇÕES BRASILEIRO BASEADAS EM VIESES COMPORTAMENTAIS ##
##
##-----------------------------------------------------------------------------------------------##

In [5]:
# Importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import statsmodels.api as sm

In [8]:
## ---------------------------------------------------------
## MarketData
## ---------------------------------------------------------
## Carregamento dos dados
## ---------------------------------------------------------
class MarketData():
    
    # Inicialiar
    def __init__(self):
        
        self.index_path = r"C:\Users\GuilhermeLuftMendesS\OneDrive - Stonecapital Consultoria e Participações Ltda\UFRGS\economatica_indices.xlsx" # Índices
        self.close_path = r"C:\Users\GuilhermeLuftMendesS\OneDrive - Stonecapital Consultoria e Participações Ltda\UFRGS\economatica_fechamento.xlsx" # Preço Fechamento
        self.volume_path = r"C:\Users\GuilhermeLuftMendesS\OneDrive - Stonecapital Consultoria e Participações Ltda\UFRGS\economatica_volume.xlsx" # Volume
        
        # Inicializar bases de dados
        self.init_data()
        
    # Inicializar dados
    def init_data(self):
        
        # Carrega as bases de dados
        self.index = self.load_data(self.index_path, names=['Data', 'IBrX'], usecols=[0,3])
        self.cdi = self.load_data(self.index_path, names=['Data', 'CDI'], usecols=[0,1])
        self.close = self.load_data(self.close_path, stockdata=True)
        self.volume = self.load_data(self.volume_path, stockdata=True)
        
        
    # Carregar dados
    # path : caminho do diretório
    # stockdata : (T|F) verdadeiro para dados de ações, falso para índice
    def load_data(self, path, stockdata=False, **kwargs):

        # Carregar o arquivo .xlsx escolhido
        data = pd.read_excel(path, index_col=0, header=0, **kwargs)

        # Corrige formatação de datas
        data.index = [dt.datetime.strptime(date, "%b-%y") for date in data.index]

        # Se forem dados de ações
        if stockdata==True:
            # Corrige nomes das colunas
            data.columns = [column[-5:] for column in data.columns]

            # Substitui valores em branco por 0
            data = data.replace(to_replace="-", value=0)

        # Retorna os dados obtidos
        return data

In [68]:
## ---------------------------------------------------------
## MarketModel
## ---------------------------------------------------------
## Classe para rodar o modelo
## ---------------------------------------------------------
class MarketModel():
    
    # Inicializar
    def __init__(self, fp, hp, data, inverse=False, short_index=False):
        
        self.fp = fp # Período de formação
        self.hp = hp # Período de permanência
        
        self.data = data # Dados (MarketData)
        self.n_stocks = 10 # Número de ações selecionadas a cada mês

        # Organizar dados
        self.setup_data()
        
        self.inverse = inverse # Se verdadeiro, performa a estratégia contrária
        self.short_index = short_index # Se verdadeiro, calcula retornos após venda do índice de referência (estratégia long-short)
        
        # Rodar
        self.run()
        
    # Organizar dados
    def setup_data(self):
        
        # Carrega as bases de dados
        self.index = self.data.index
        self.close = self.data.close
        self.volume = self.data.volume
        self.cdi = self.data.cdi
        
    # Normalizar portfólios
    def normalize(self, portfolio):
        
        d = len(portfolio) - len(self.portfolio)
        
        return portfolio[d:] / portfolio.iloc[d].values[0]
    
    
    # Função que retorna as ações que estão no primeiro decil de volume negociado no período i
    def top_decile(self,i):

        # Calcula o volume necessário para estar no primeiro decil de ações mais negociadas neste período
        decile_cut = np.percentile(self.volume.iloc[i], q=90)

        # Verifica quais ações negociaram pelo menos este valor de corte no período e retorna
        return self.close.loc[:, self.volume.ge(decile_cut).iloc[i].values]
    

    # Função que constrói o histórico de posições do portfólio
    def build_portfolio(self):

        # Inicia com um array vazio
        stocks = []

        # Cria outro array para salvar as ações sem overlap de períodos
        stocks_wo_overlap = []

        # Repetir do período fp até o último período
        for i in range(self.fp, len(self.volume)):

            # Define a amostra a partir do decil de ações negociadas com maior volume
            sample = self.top_decile(i)

            # Considera apenas as ações que foram negociadas em todos os meses no período i a i-fp
            sample = sample.loc[:, (sample.iloc[[i,i-self.fp]] != 0).all(axis=0)]

            # Considera o retorno destas ações no período
            sample = sample.iloc[i] / sample.iloc[i-self.fp].values - 1

            # Ordena os retornos em ordem crescente
            sample = pd.DataFrame(columns=['Retorno'], index=sample.index, data=sample.values)
            sample = sample.sort_values(by="Retorno")

            # W deste período é o conjunto das 'n' últimas ações (maior retorno passado), e L as 'n' primeiras (menor retorno)
            # Define entre retornar W ou L.
            if self.inverse==True:
                new_stocks = sample.index[:self.n_stocks].values
            else:
                new_stocks = sample.index[-self.n_stocks:].values

            # Adiciona as ações deste período ao array sem overlap
            stocks_wo_overlap.append(new_stocks)

            # Se já houver no mínimo hp portfólios formados
            if len(stocks_wo_overlap) >= self.hp:

                # Constrói o array para adicionar as ações deste período com overlap dos períodos anteriores, com base no
                # período de permanência (hp)
                add_stocks = []

                for t in range(0,self.hp):

                    # Adiciona as ações do período i-t
                    add_stocks = [*add_stocks, *stocks_wo_overlap[i-self.fp-t]]

                # Adiciona todas as ações consideradas a 'stocks'
                stocks.append(add_stocks)

        # Converte em DataFrame para manipulação
        stocks = pd.DataFrame(data=stocks, index=self.close.index[(self.fp+self.hp-1):])

        return stocks
    
    # Custo de transação
    def tc(self, from_p, to_p):

        # Diferença entre o peso atual dos ativos no portfólio e o peso desejado após transações
        wdif = from_p - to_p.values
        
        s1 = wdif.where(wdif >= 0).sum() # Soma do excesso de participação dos ativos que serão vendidos
        s2 = to_p.where(wdif >= 0).sum() # Soma do peso desejado dos ativos que serão vendidos
        
        t = 0.005 # Taxa de corretagem
        ctc = 2 * t / (1+t) # Constante proporcional a t
        
        # Fórmula proporcional à Constante e S1
        s = s1 / (1 - s2 * ctc) * ctc
        
        return s

    # Peso (%) dos ativos no portfólio ao longo do tempo
    def weights(self):
        
        # Inicia um DF em branco
        df = pd.DataFrame(index=self.close.index, data=0.0, columns=self.close.columns)

        # Conta o total de posições (não únicas) no período
        c = self.portfolio.count(axis=1)
        
        # Para cada período
        for y in range(0,len(self.portfolio)):
            
            # Seleciona este período
            idx = self.portfolio.index[y]

            # Para cada ação no portfólio
            for x in self.portfolio.columns:        

                # Aumenta o peso desta ação no período correspondentemente
                ticker = self.portfolio.at[idx, x]
                if ticker != None:                    
                    df.at[idx, ticker] = df.at[idx, ticker] + 1 / c[y]

        # Corrige o tamanho do DF
        dif = len(df) - len(self.portfolio)
        df = df[dif:]
        
        # Retorna
        return df
    
    
    # Função que retorna o histórico (evolução de $1 investido) de um portfólio, em que este é definido a partir de
    # um DataFrame com o conjunto de ações daquele portfólio em cada período do tempo
    def history(self):

        # Inicializa as variáveis
        money = 1
        money_history = [money]
        portfolio = self.portfolio
        df = self.weights() # Peso dos ativos no portfólio

        # Para cada período,
        for k in range(0, len(portfolio)-1):

            # Inicializa as variáveis
            factor = 0
            used = []
            sumw = 0

            # Para cada ação no portfólio neste período
            for ticker in portfolio.iloc[k].values:
                
                # Se o ticker for válido e ainda não tiver sido utilizado
                if type(ticker) == str and not ticker in used:

                    # Adiciona aos tickers já utilizados
                    used.append(ticker)
                    
                    # Define o período de análise
                    period = k + self.fp + self.hp
                
                    ap = self.close[[ticker]].iloc[period].values[0] # Preço atual
                    lp = self.close[[ticker]].iloc[period-1].values[0] # Preço anterior

                    # Se houver dados de preço de fechamento desta ação no período atual e no período anterior
                    if ap != 0 and lp != 0:

                        # Calcula o peso do ativo no portfólio
                        w = df.at[self.portfolio.index[k], ticker]

                        # Adiciona ao fator de rentabilidade o retorno ponderado desta ação no período
                        factor = factor + (ap / lp - 1) * w

                        # Aumenta a contagem do peso total utilizado
                        sumw = sumw + w

            # Corrige proporcionalmente a rentabilidade do portfólio caso o peso dos ativos no período não seja 100%,
            # por conta de algum dado que esteja faltando
            if sumw != 0:
                factor = factor / sumw
            factor = (factor + 1)
            
            # Retira da rentabilidade os custos de transação:

            # Se este for o primeiro período
            if k==0:
                tc = 0.005 # Custo inicial

            # Caso contrário
            else:

                from_p = df.loc[self.portfolio.index[k-1],:] # Portfólio no mês anterior
                to_p = df.loc[self.portfolio.index[k],:] # Portfólio neste mês
                tc = self.tc(from_p, to_p) # Custo de rebalanceamento
                
            # Retira o custo da rentabilidade
            factor = factor * (1 - tc)


            # Se estiver vendido no índice
            if self.short_index == True:

                # Calcula a rentabilidade do índice no mês
                rm = self.index.iloc[period].values[0] / self.index.iloc[period-1].values[0] - 1

                # Calcula a rentabilidade do CDI no mês
                rf = self.cdi.iloc[period].values[0] / self.cdi.iloc[period-1].values[0] - 1

                # Calcula o custo de transção adicional (taxa de aluguel do índice)
                tc_short = (1.02 ** (1/12)) - 1

                # Calcula o custo de ficar vendido
                sc = rf - rm - tc_short

                # Retira o custo da rentabilidade
                factor = factor * (1 + sc)

            # Atualiza o histórico de $
            money = money * factor
            money_history.append(money)

        # Converte o histórico em DataFrame e retorna
        money_history = pd.DataFrame(data=money_history, index=self.close.index[(self.fp+self.hp-1):])

        return money_history

    
    # Função que retorna os dados necessários para rodar as regressões
    def regression_data(self):

        # Calcula a taxa de retorno livre de risco
        lCDI = [np.log(x[0]) for x in self.n_cdi.values]
        lCDI = pd.DataFrame(index=self.n_cdi.index, data=lCDI)
        lCDI = lCDI - lCDI.shift(1)

        # Calcula o excesso de retorno do portfolio em relação ao ativo livre de risco
        h = self.portfolio_history
        lRET = [np.log(x[0]) for x in h.values]
        lRET = pd.DataFrame(index=h.index, data=lRET)
        lRET = lRET - lRET.shift(1)

        exc_ret = lRET - lCDI
        exc_ret.columns = ['Portfolio']

        # Calcula o prêmio de risco de mercado
        lMKT = [np.log(x[0]) for x in self.n_index.values]
        lMKT = pd.DataFrame(index=self.n_index.index, data=lMKT)
        lMKT = lMKT - lMKT.shift(1)
        erp = lMKT - lCDI
        erp = sm.add_constant(erp)
        erp.columns = ['Alpha', 'Beta']

        return exc_ret[1:], erp[1:]
    
    # Função que calcula o retorno total anualizado do portfólio
    def ann_return(self, port):
        
        ann_return = ((port.iloc[len(port)-1].values[0]) ** (12/len(port)) - 1) * 100
        
        return ann_return
    
    
    # Função para plotagem de resultados
    def plot(self, portfolios, names, dtype=['-', '--']):
        
        # Inicializa o gráfico
        fig, ax = plt.subplots(figsize=(11,6))
        plt.yscale('log')

        # Para cada portfolio
        for p in range(0,len(portfolios)):
            
            # Plota o gráfico
            port = portfolios[p]        
            ax.plot(port, dtype[p], label=names[p], color='black', linewidth=1)
                
        # Detalhes do gráfico
        ax.grid(color='lightgrey')
        ax.legend()

        # Mostra o gráfico
        plt.show()
        
        
    # Função para executar o programa
    def run(self):

        # Constrói o portfólio
        self.portfolio = self.build_portfolio()
        self.portfolio_history = self.history()
        
        # Índices normalizados
        self.n_cdi = self.normalize(self.cdi)
        self.n_index = self.normalize(self.index)

In [10]:
## ---------------------------------------------------------
## Analysis
## ---------------------------------------------------------
## Procedimentos para análise de regressões
## ---------------------------------------------------------
class Analysis():

    # Inicializar
    def __init__(self, mm):

        self.exc_ret, self.erp = mm.regression_data()

    # Parâmetros de interesse da regressão
    def params(self):

        # Roda o modelo MQO
        model = sm.OLS(self.exc_ret, self.erp)
        results = model.fit()

        # Coleta os parâmetros
        alpha = results.params[['Alpha']][0]
        beta = results.params[['Beta']][0]
        p_alpha = results.pvalues[['Alpha']][0]

        # Retorna parâmetros
        return (alpha, p_alpha, beta)


In [52]:
## ---------------------------------------------------------
## Export
## ---------------------------------------------------------
## Exportação dos dados para um arquivo .txt
## ---------------------------------------------------------
class Export():

    # Inicializar
    def __init__(self):

        # Carrega os dados
        self.data = MarketData()

        # Rodar
        self.run()
    
    # Rodar
    def run(self):

        count=0
        fps = [12,24,36,48,60]
        hps = [12,24,36,48,60]

        # Para cada combinação de fp e hp
        for fp in fps:
            for hp  in hps:

                # Cria os modelos
                model = [MarketModel(fp,hp, self.data, inverse=False, short_index=False),
                MarketModel(fp,hp, self.data, inverse=False, short_index=True),
                MarketModel(fp,hp, self.data, inverse=True, short_index=False),
                MarketModel(fp,hp, self.data, inverse=True, short_index=True)]

                names = ['WLO', 'WLS', 'LLO', 'LLS']

                # Para cada modelo
                for i in range(4):

                    # Seleciona
                    m = model[i]

                    name = names[i] + str(fp) + '.' + str(hp) # Nome
                    h = m.portfolio_history # Histórico
                    an = Analysis(m) # Análise
                    alpha, p_alpha, beta = an.params() # Parâmetros

                    # Seleciona o arquivo para output
                    with open('output.txt','a') as file:
                        
                        # Exporta os dados
                        # nome;alpha;p(alpha);beta;ann_return;ann_return_index;ann_return_cdi
                        file.write('\n%s;%f;%f;%f;%f;%f;%f' % (name, alpha, p_alpha, beta, mm.ann_return(h), mm.ann_return(mm.n_index), mm.ann_return(mm.n_cdi)))

                count = count+1
                print("%d/%d" % (count, 25))

    